# Equity Research

An equity research analyst is a professional who evaluates public companies and provides recommendations on whether to buy, sell, or hold their stocks.

**Buy-Side Analysts** work for institutional investors such as hedge funds ...
- [ ] Please complete a bit more of a blurb here

**Sell-side Analysts** work for brokerage firms and investment banks ...
- [ ] Please complete a bit more of a blurb here

Today we are going to put ourselves in the shoes of a **Sell-Side** Equity Analyst. 

We are going to look at some of the tools we can use in Python to assist in building an equity research report for publication.
We are not so much focussed on the process of formulating the analysis such as building a DCF.
Rather we want to learn how to use the tools that can automate some of the work we may do as an analyst.

Step 1 will be to create a few handy functions

Step 2 will be to pull in some live data from an application programming interface (API)

- [ ] Please describe in successive slide fragments all the things we are going to do in the lecture

- [ ] Please describe in successive slide fragments all the things we are going to do in the lecture

# Create some handy functions
Here we are going to create some functions that we can call to do some calculations

In [ ]:
# Function to calculate the P/E ratio of a stock
def calculate_pe_ratio(price, earnings):
    return price / earnings

In [ ]:
# Function to calculate the dividend yield of a stock
def calculate_dividend_yield(dividend, price):
    return dividend / price

In [ ]:
# Function to calculate the earnings per share of a stock
def calculate_eps(net_income, shares_outstanding):
    return net_income / shares_outstanding

In [ ]:
# Function to calculate ROE of a stock
def calculate_roe(net_income, equity):
    return net_income / equity

In [ ]:
# Function to calculate ROA of a stock
def calculate_roa(net_income, total_assets):
    return net_income / total_assets

# Application Programming Interfaces (APIs)

- [ ] Convert my slides from last year into Jupyter Slide Show cells (you may need to create .jpg files for the diagrams and render them in the slideshow)

In [ ]:
# Use the joke of the day API and the weather API examples from last years slides to show how to use APIs in Python 
# these are contained in API_examples.ipynb

yfinance is a Python library that allows you to download financial data from Yahoo Finance. It provides a simple and convenient way to access historical stock prices, dividend data, and other financial data for publicly traded companies. With yfinance, you can easily download data for multiple stocks at once and perform analysis on the data using popular Python libraries like Pandas and Matplotlib.

In [ ]:
# pull some data on an australian stock from yfinance
# and feed this data into the functions above to calculate some ratios etc


In [ ]:
# use the read_html() function from pandas to scrape some data from a website that is relevant to the stock
# again, you can refer to an example from API_examples.ipynb


In [ ]:
# use ipywidgets to create an interactive widget that allows the user to input the stock code 
# this will then pull the data from yfinance and run the functions above to calculate the ratios

In [18]:
import ipywidgets as widgets
from IPython.display import display

# Create a text input widget
stock_code_input = widgets.Text(
    value='',
    placeholder='Stock code',
    description='Stock code:',
    disabled=False
)

# Create a button widget
submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

# Define the function to handle button click
def on_button_click(b):
    stock_code = stock_code_input.value
    print(f'Entered Stock Code: {stock_code}')

# Attach the function to the button click event
submit_button.on_click(on_button_click)

# Display the widgets
display(stock_code_input, submit_button)

Text(value='', description='Stock code:', placeholder='Stock code')

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Click me')

In [19]:
# print the user input
print(f'User entered stock code: {stock_code_input.value}')

User entered stock code: BHP


In [ ]:
# code to read commentary from a text file into a string for later inclusion in the analyst report
# also read from a csv file into a pandas dataframe (using pandas.read_csv(file_path)) for later inclusion in the analyst report

In [ ]:
# show the students how to create a pdf file

from fpdf import FPDF

# Create instance of FPDF class
pdf = FPDF()

# Add a page
pdf.add_page()

# Set font
pdf.set_font("Arial", size=12)

# Add a cell
pdf.cell(200, 10, txt="Welcome to Python PDF creation!", ln=True, align='C')

# Save the PDF with name .pdf
pdf.output("simple_demo.pdf")

print("PDF created successfully!")

In [17]:
# get DCF output from an excel file

import pandas as pd

# Read the excel file
df = pd.read_excel('DCF_test.xlsx', sheet_name="Sheet1", usecols='A:L', skiprows=0, nrows=3)

print (df)

              Year         2025         2026         2027         2028  \
0         Cashflow  2208.234503  8409.837696  3066.882847  2526.980787   
1  Discount factor     0.952381     0.907029     0.863838     0.822702   

          2029         2030         2031         2032        2033  \
0  5670.492956  6511.520390  3202.663033  1241.044536  718.228398   
1     0.783526     0.746215     0.710681     0.676839    0.644609   

          2034         2035  
0  4387.708928  7484.910830  
1     0.613913     0.584679  
